In [2]:
from qutip import *
from math import sqrt, cos, sin
import numpy as np
from gen_state import make_W_list
from rho_methods import compute_witnesses, get_rho
import matplotlib.pyplot as plt
from tqdm import tqdm

------ 1 -------
Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.74076   +0.00804567j]
 [0.        -0.66768663j]
 [0.00804567-0.07307337j]
 [0.        +0.j        ]]
Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.54879011+0.j         -0.00537199+0.49459555j  0.00537199+0.05419456j
   0.        +0.j        ]
 [-0.00537199-0.49459555j  0.44580544+0.j          0.04879011-0.00537199j
   0.        +0.j        ]
 [ 0.00537199-0.05419456j  0.04879011+0.00537199j  0.00540445+0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]]
0.0
------ 2 -------
Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[ 0.        +0.j        ]
 [ 0.07307337+0.00804567j]
 [ 0.66768663+0.j        ]
 [-0.00804567+0.74076j   ]]
Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.  

In [3]:
H = basis(2,0) # ket 1
V = basis(2,1) # ket 2
D = 1/sqrt(2) * (H + V)
A = 1/sqrt(2) * (H - V)
R = 1/sqrt(2)*(H + 1j*V)
L = 1/sqrt(2)*(H - 1j*V)

In [10]:
def pauli_state(W_mat):
    """ Checks which pauli matrix combos make up the witness, W_mat

        0 - IxI 4 - XxI 8 - YxI 12 - ZxI
        1 - IxX 5 - XxX 9 - YxX 13 - ZxX
        2 - IxY 6 - XxY 10 - YxY 14 - ZxY
        3 - IxZ 7 - XxZ 11 - YxZ 15 - ZxZ
    """

    II = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]]).flatten()
    IX = np.array([[0,1,0,0],[1,0,0,0],[0,0,0,1],[0,0,1,0]]).flatten()
    IY = np.array([[0,-1j,0,0],[1j,0,0,0],[0,0,0,-1j],[0,0,1j,0]]).flatten()
    IZ = np.array([[1,0,0,0],[0,-1,0,0],[0,0,1,0],[0,0,0,-1]]).flatten()

    XI = np.array([[0,0,1,0],[0,0,0,1],[1,0,0,0],[0,1,0,0]]).flatten()
    XX = np.array([[0,0,0,1],[0,0,1,0],[0,1,0,0],[1,0,0,0]]).flatten()
    XY = np.array([[0,0,0,-1j],[0,0,1j,0],[0,-1j,0,0],[1j,0,0,0]]).flatten()
    XZ = np.array([[0,0,1,0],[0,0,0,-1],[1,0,0,0],[0,-1,0,0]]).flatten()

    YI = np.array([[0,0,-1j,0],[0,0,0,-1j],[1j,0,0,0],[0,1j,0,0]]).flatten()
    YX = np.array([[0,0,0,-1j],[0,0,1j,0],[0,1j,0,0],[1j,0,0,0]]).flatten()
    YY = np.array([[0,0,0,-1],[0,0,1,0],[0,1,0,0],[-1,0,0,0]]).flatten()
    YZ = np.array([[0,0,-1j,0],[0,0,0,1j],[1j,0,0,0],[0,-1j,0,0]]).flatten()

    ZI = np.array([[1,0,0,0],[0,1,0,0],[0,0,-1,0],[0,0,0,-1]]).flatten()
    ZX = np.array([[0,1,0,0],[1,0,0,0],[0,0,0,-1],[0,0,-1,0]]).flatten()
    ZY = np.array([[0,-1j,0,0],[1j,0,0,0],[0,0,0,1j],[0,0,-1j,0]]).flatten()
    ZZ = np.array([[1,0,0,0],[0,-1,0,0],[0,0,-1,0],[0,0,0,1]]).flatten()
    
    W_flat = W_mat.flatten() # for some reason this doesn't work as desired ... 4x4 matrix
    
    pauli_mats =  np.array([II,IX,IY,IZ,XI,XX,XY,XZ,YI,YX,YY,YZ,ZI,ZX,ZY,ZZ])
    contains_pauli = [False]*16

    for i in range(len(pauli_mats)):
        prod = np.dot(W_flat,pauli_mats[i])
        if prod != 0:
            contains_pauli[i] = True
    return contains_pauli

In [12]:
W_state = tensor(H,H) + 1j*tensor(H,V) + tensor(V,H) -1j*tensor(V,V)
W = partial_transpose(W_state*W_state.dag(), [0,1])
W_mat = W.full()

pauli_mats = pauli_state(W_mat)
# II,IX,IY,IZ,XI,XX,XY,XZ,YI,YX,YY,YZ,ZI,ZX,ZY,ZZ
# [True, False, False, False, False, False, False, True, False, True, False, False, False, False, True, False]
print(pauli_mats)

[True, False, False, False, False, False, False, True, False, True, False, False, False, False, True, False]


In [9]:
num_intervals = 10

# Generate mesh grids for each parameter
alpha_L, beta_L, theta_L, phi_L, chi_L = np.meshgrid(np.linspace(0, np.pi, num_intervals),
                                                      np.linspace(0, 2*np.pi, num_intervals),
                                                      np.linspace(0, np.pi, num_intervals),
                                                      np.linspace(0, 2*np.pi, num_intervals),
                                                      np.linspace(0, 2*np.pi, num_intervals),
                                                      indexing='ij')
# Flatten mesh grids
alpha_vals = alpha_L.flatten()
beta_vals = beta_L.flatten()
theta_vals = theta_L.flatten()
phi_vals = phi_L.flatten()
chi_vals = chi_L.flatten()

for alpha, beta, theta, phi, chi in tqdm(zip(alpha_vals, beta_vals, theta_vals, phi_vals, chi_vals)):
    # Calculate entangled state
    ent_state = tensor(np.cos(theta)*H + np.sin(theta)*np.exp(1j*phi)*V,
                       np.cos(alpha)*H + np.sin(alpha)*np.exp(1j*beta)*V) + \
                np.exp(1j*chi) * tensor(-np.exp(-1j*phi)*np.sin(theta)*H + np.cos(theta)*V,
                                         -np.exp(-1j*beta)*np.sin(alpha)*H + np.cos(alpha)*V)
    rho = ket2dm(ent_state)
    exp_val = (W*rho).tr()
    if np.real(exp_val) < 0:
        print(alpha, beta, theta, phi, chi)

210it [00:00, 650.42it/s]

0.0 0.0 0.3490658503988659 2.0943951023931953 0.6981317007977318
0.0 0.0 0.3490658503988659 2.0943951023931953 1.3962634015954636
0.0 0.0 0.3490658503988659 2.792526803190927 0.6981317007977318
0.0 0.0 0.3490658503988659 2.792526803190927 1.3962634015954636
0.0 0.0 0.3490658503988659 2.792526803190927 2.0943951023931953
0.0 0.0 0.3490658503988659 3.490658503988659 0.6981317007977318
0.0 0.0 0.3490658503988659 3.490658503988659 1.3962634015954636
0.0 0.0 0.3490658503988659 3.490658503988659 2.0943951023931953
0.0 0.0 0.3490658503988659 3.490658503988659 2.792526803190927
0.0 0.0 0.3490658503988659 4.1887902047863905 1.3962634015954636
0.0 0.0 0.3490658503988659 4.1887902047863905 2.0943951023931953
0.0 0.0 0.3490658503988659 4.1887902047863905 2.792526803190927
0.0 0.0 0.6981317007977318 2.0943951023931953 0.0
0.0 0.0 0.6981317007977318 2.0943951023931953 0.6981317007977318
0.0 0.0 0.6981317007977318 2.0943951023931953 1.3962634015954636
0.0 0.0 0.6981317007977318 2.0943951023931953 6.2

340it [00:00, 630.49it/s]

0.0 0.0 0.6981317007977318 4.1887902047863905 1.3962634015954636
0.0 0.0 0.6981317007977318 4.1887902047863905 2.0943951023931953
0.0 0.0 0.6981317007977318 4.1887902047863905 2.792526803190927
0.0 0.0 0.6981317007977318 4.1887902047863905 3.490658503988659
0.0 0.0 1.0471975511965976 2.0943951023931953 0.0
0.0 0.0 1.0471975511965976 2.0943951023931953 0.6981317007977318
0.0 0.0 1.0471975511965976 2.0943951023931953 5.585053606381854
0.0 0.0 1.0471975511965976 2.0943951023931953 6.283185307179586
0.0 0.0 1.0471975511965976 2.792526803190927 0.0
0.0 0.0 1.0471975511965976 2.792526803190927 0.6981317007977318
0.0 0.0 1.0471975511965976 2.792526803190927 1.3962634015954636
0.0 0.0 1.0471975511965976 2.792526803190927 2.0943951023931953
0.0 0.0 1.0471975511965976 2.792526803190927 6.283185307179586
0.0 0.0 1.0471975511965976 3.490658503988659 0.6981317007977318
0.0 0.0 1.0471975511965976 3.490658503988659 1.3962634015954636
0.0 0.0 1.0471975511965976 3.490658503988659 2.0943951023931953
0.0

524it [00:00, 583.51it/s]

...

99794it [02:37, 602.65it/s]

3.141592653589793 6.283185307179586 2.443460952792061 0.0 1.3962634015954636
3.141592653589793 6.283185307179586 2.443460952792061 0.0 2.0943951023931953
3.141592653589793 6.283185307179586 2.443460952792061 0.0 2.792526803190927
3.141592653589793 6.283185307179586 2.443460952792061 0.6981317007977318 1.3962634015954636
3.141592653589793 6.283185307179586 2.443460952792061 0.6981317007977318 2.0943951023931953
3.141592653589793 6.283185307179586 2.443460952792061 0.6981317007977318 2.792526803190927
3.141592653589793 6.283185307179586 2.443460952792061 0.6981317007977318 3.490658503988659
3.141592653589793 6.283185307179586 2.443460952792061 1.3962634015954636 2.0943951023931953
3.141592653589793 6.283185307179586 2.443460952792061 1.3962634015954636 2.792526803190927
3.141592653589793 6.283185307179586 2.443460952792061 1.3962634015954636 3.490658503988659
3.141592653589793 6.283185307179586 2.443460952792061 4.886921905584122 0.0
3.141592653589793 6.283185307179586 2.443460952792061 

99920it [02:38, 616.48it/s]

3.141592653589793 6.283185307179586 2.792526803190927 0.6981317007977318 1.3962634015954636
3.141592653589793 6.283185307179586 2.792526803190927 0.6981317007977318 2.0943951023931953
3.141592653589793 6.283185307179586 2.792526803190927 0.6981317007977318 2.792526803190927
3.141592653589793 6.283185307179586 2.792526803190927 1.3962634015954636 2.0943951023931953
3.141592653589793 6.283185307179586 2.792526803190927 1.3962634015954636 2.792526803190927
3.141592653589793 6.283185307179586 2.792526803190927 4.886921905584122 0.6981317007977318
3.141592653589793 6.283185307179586 2.792526803190927 4.886921905584122 1.3962634015954636
3.141592653589793 6.283185307179586 2.792526803190927 5.585053606381854 0.0
3.141592653589793 6.283185307179586 2.792526803190927 5.585053606381854 0.6981317007977318
3.141592653589793 6.283185307179586 2.792526803190927 5.585053606381854 1.3962634015954636
3.141592653589793 6.283185307179586 2.792526803190927 5.585053606381854 2.0943951023931953
3.141592653

100000it [02:38, 632.44it/s]
